In [1]:
%display latex

In [2]:
import algorithm_states

In [3]:
k = QQ.algebraic_closure()
q0 = matrix([
    [0, 0, 0, 1/2],
    [0, 0, -1/2, 0],
    [0, -1/2, 0, 0],
    [1/2, 0, 0, 0],
]).change_ring(k)

Take $ m \in \mathop{SO}(q_0)(k) $.

In [4]:
m0 = matrix([
    [0, 0, -1, 0],
    [0, 0, 0, 1],
    [-1, 0, 0, 0],
    [0, 1, 0, 0]
]); (m0, m0.transpose() * q0 * m0 - q0, m0.determinant())

(
[ 0  0 -1  0]  [0 0 0 0]   
[ 0  0  0  1]  [0 0 0 0]   
[-1  0  0  0]  [0 0 0 0]   
[ 0  1  0  0], [0 0 0 0], 1
)

In [5]:
state = algorithm_states.SL2SL2State(m0)

First of all, if $ m_{11} = 0 $ and $ m_{21} = 0 $, we must have $ m_{31} \not = 0 $ or $ m_{41} \not = 0 $. We add $ m_{31} $ to $ m_{11} $ and $ m_{41} $ to $ m_{21} $ to have $ m_{11} \not = 0 $ or $ m_{21} \not = 0 $.

In [6]:
if state.m[0, 0] == 0 and state.m[0, 1] == 0:
    state.add_row(2, 0)

[-1  0 -1  0]
[ 0  1  0  1]
[-1  0  0  0]
[ 0  1  0  0]

If $ m_{21} = 0 $, we add $ m_{11} $ to $ m_{21} $ to have $ m_{21} \not = 0 $.

In [7]:
if state.m[1, 0] == 0:
    state.add_row(0, 1)

[-1  0 -1  0]
[-1  1 -1  1]
[-1  0  0  0]
[-1  1  0  0]

We add $ \frac{1 - m_{11}}{m_{21}} $ times $ m_{21} $ to $ m_{11} $ to have $ m_{11} = 1 $.

In [8]:
state.add_row(1, 0, scale = (1 - state.m[0, 0]) / state.m[1, 0])

[ 1 -2  1 -2]
[-1  1 -1  1]
[ 1 -2  0  0]
[-1  1  0  0]

We now subtract $ m_{21} $ times $ m_{11} $ from $ m_{21} $, and $ m_{31} $ times $ m_{11} $ from $ m_{31} $. Then $ m_{21} = m_{31} = 0 $.

In [9]:
state.add_row(0, 1, scale = -state.m[1, 0])

[ 1 -2  1 -2]
[ 0 -1  0 -1]
[ 1 -2  0  0]
[ 0 -1  0  0]

In [10]:
state.add_row(0, 2, scale = -state.m[2, 0])

[ 1 -2  1 -2]
[ 0 -1  0 -1]
[ 0  0 -1  2]
[ 0  0  0  1]

Since $ m \in \mathop{SO}(q_0)(k) $, for $ i \leq 3 $, the inner product between columns $ 1 $ and $ i $ is $ 0 $, so $ m_{4i} = 0 $. Also, the inner product between columns $ 1 $ and $ 4 $ is $ \frac{1}{2} $, so $ m_{44} = 1 $.

We can now subtract $ m_{34} $ times $ m_{44} $ from $ m_{34} $ and $ m_{24} $ times $ m_{44} $ from $ m_{24} $ to make sure $ m_{24} = m_{34} = 0 $.

In [11]:
state.add_row(3, 2, scale = -state.m[2, 3])

[ 1  0  1  0]
[ 0 -1  0 -1]
[ 0  0 -1  0]
[ 0  0  0  1]

In [12]:
state.add_row(3, 1, scale = -state.m[1, 3])

[ 1  0  0  0]
[ 0 -1  0  0]
[ 0  0 -1  0]
[ 0  0  0  1]

For $ 2 \leq i \leq 4 $, the inner product between columns $ i $ and $ 4 $ is $ 0 $, so $ m_{4i} = 0 $.

Then the equation, given at the end of Appendix A to cut out $ \mathop{SO}(q_0) $ inside $ \mathop{O}(q_0) $, gives $ m_{23} m_{32} = 0 $. Also, since the inner product between the columns $ 2 $ and $ 3 $ is $ \frac{1}{2} $, we have $ m_{22} m_{33} = 1 $. Now for $ 2 \leq i \leq 3 $, since the inner product of column $ i $ with itself is $ 0 $ for all columns, we have $ m_{2i} m_{3i} = 0 $ and this gives $ m_{23} = m_{32} = 0 $. Then we have $ m_{33} = \frac{1}{m_{22}} $.

Since $ k $ is quadratically closed, there exists $ \mu \in k $ such that $ \mu^2 = m_{22} $. Then we can lift $ m $ to $ \left( \left( \begin{smallmatrix} \mu & 0 \\ 0 & \frac{1}{\mu} \end{smallmatrix} \right), \left( \begin{smallmatrix} \frac{1}{\mu} & 0 \\ 0 & \mu \end{smallmatrix} \right) \right) \in (\mathop{SL}_2 \times \mathop{SL}_2)(k) $. Multiplying by its inverse concludes the proof.

In [13]:
mu = k(sqrt(state.m[1, 1]))
state.step((diagonal_matrix([1/mu, mu]), diagonal_matrix([mu, 1/mu])))

[1 0 0 0]
[0 1 0 0]
[0 0 1 0]
[0 0 0 1]

Then inverting the two matrices gives the lift for $ m $.

In [14]:
(lift1, lift2) = state.t; (lift1, lift2)

(
[ 0 -I]  [-I  0]
[-I  0], [ 0  I]
)

In [15]:
(lift1.inverse(), lift2.inverse())

(
[0 I]  [ I  0]
[I 0], [ 0 -I]
)

In [16]:
(m0, state.project(lift1.inverse(), lift2.inverse()))

(
[ 0  0 -1  0]  [ 0  0 -1  0]
[ 0  0  0  1]  [ 0  0  0  1]
[-1  0  0  0]  [-1  0  0  0]
[ 0  1  0  0], [ 0  1  0  0]
)